<a href="https://colab.research.google.com/github/Darshit1617/email_spam_classifier/blob/main/email_spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Install the Required Libraries**

In [26]:
!pip install pandas numpy scikit-learn streamlit pyngrok

# **2. Import Essential Python Modules**


In [27]:
import pandas as pd
import numpy as np
import os
import pickle
from threading import Thread
from pyngrok import ngrok
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

# **3. Load the Dataset**
dataset is preprocessed.

In [29]:
# Enter dataset location where your 'emails.csv' is stored
dataset_location = "enter dataset location"
df = pd.read_csv('/content/emails.csv')
print("Dataset loaded. Shape:", df.shape)

Dataset loaded. Shape: (5172, 3002)


# **4. Feature extraction & Data Preparation**

In [30]:
# Assuming first column is email name and last column is the label
X = df.iloc[:, 1:-1]  # Feature columns (3000 common words)
y = df.iloc[:, -1]    # Labels (spam=1, ham=0)

# Split data into training and testing sets (80% train / 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training and testing sets created.")

Training and testing sets created.


# **5. Model Training: Multinomial Naive Bayes**

In [31]:
# Initialize and train the Multinomial Naive Bayes classifier
model = MultinomialNB()
model.fit(X_train, y_train)
print("Model training complete.")

Model training complete.


# **6. Model Evaluation**

In [32]:
# Perform predictions and evaluate the model's accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)

Model Accuracy: 0.9545893719806763


# **7. Build a CountVectorizer for New Email Inputs**

In [33]:
# Create a vocabulary list from the training feature columns
vocabulary = list(X.columns)

# Initialize CountVectorizer with the given vocabulary
vectorizer = CountVectorizer(vocabulary=vocabulary)
print("CountVectorizer is ready with the provided vocabulary.")

CountVectorizer is ready with the provided vocabulary.


# **8. Prediction Function for a New Email**

In [34]:
# Function to predict if an email is spam or ham (not spam)
def predict_email(email_text):
    # Transform the raw email into a count vector
    email_vector = vectorizer.transform([email_text])
    # Make a prediction using the trained model
    prediction = model.predict(email_vector)
    return prediction[0]

# Testing the prediction function with an example email
email_example = "Congratulations! You’ve won a FREE iPhone! Click here to claim your prize now before it expires in 24 hours!!! Don’t miss out on this amazing deal – act fast!"
result = predict_email(email_example)
if result == 1:
    print("The email is SPAM!")
else:
    print("The email is NOT SPAM (Ham).")

The email is SPAM!


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


# **9. Save the Trained Model and Vectorizer**

In [10]:
# Save the model
with open("spam_classifier.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

# Save the vectorizer
with open("vectorizer.pkl", "wb") as vec_file:
    pickle.dump(vectorizer, vec_file)

print("Model and vectorizer have been saved.")


Model and vectorizer have been saved.


# **10. Streamlit App for Deployment**

In [36]:
%%writefile app.py
import streamlit as st
import pickle
from sklearn.feature_extraction.text import CountVectorizer


# Load the pre-trained model and vectorizer
model = pickle.load(open("spam_classifier.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

st.title("Email Spam Classifier")

email_input = st.text_area("Enter your email text:")

# if st.button("Predict"):
#     email_vect = vectorizer.transform([email_input])
#     prediction = model.predict(email_vect)
#     if prediction[0] == 1:
#         st.write("⚠️ Warning: This email is classified as spam!")
#     else:
#         st.write("✅ This email is not spam (ham) and appears safe!")

if st.button("Predict"):
    if email_input:
        email_vect = vectorizer.transform([email_input])
        prediction = model.predict(email_vect)
        if prediction[0] == 1:
          st.write("⚠️ Warning: This email is classified as spam!")
        else:
          st.write("✅ This email is not spam (ham) and appears safe!")
    else:
        st.write("Please enter an email text.")

Overwriting app.py


In [37]:
def run_streamlit():
    # Change the port if 8501 is already in use or if you prefer another port
    os.system('streamlit run /content/app.py --server.port 8501')

In [38]:
# Start a thread to run the Streamlit app
thread = Thread(target=run_streamlit)
thread.start()

*ngrock token*

In [39]:
ngrok.set_auth_token('_____ngrock_Auth_token_paste_here_______')

In [41]:
# Open a tunnel to the streamlit port 8501
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Your Streamlit app is live at:', public_url)

Your Streamlit app is live at: NgrokTunnel: "https://b9c2-35-231-254-193.ngrok-free.app" -> "http://localhost:8501"
